In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline
# from matplotlib.pylab import rcParams

mig = pd.read_csv('/Users/xiaotingzhong/Documents/Matlab/Grain Tracking/190408_features_mig.txt')
mig_sign = pd.read_csv('/Users/xiaotingzhong/Documents/Matlab/Grain Tracking/190408_features_mig_signs.txt')
energy_grad = pd.read_csv('/Users/xiaotingzhong/Documents/Matlab/Grain Tracking/190408_features_energygrad_new.txt')
geo_topo = pd.read_csv('/Users/xiaotingzhong/Documents/Matlab/Grain Tracking/190408_features_geo_topo.txt')
other = pd.read_csv('/Users/xiaotingzhong/Documents/Matlab/Grain Tracking/190408_features_otherinfo.txt')

mig.columns = mig.columns.str.strip()
mig_sign.columns = mig_sign.columns.str.strip()
energy_grad.columns = energy_grad.columns.str.strip()
geo_topo.columns = geo_topo.columns.str.strip()
other.columns = other.columns.str.strip()

In [ ]:
def plotConfusionMatrix(y_true, y_pred, classes,
                          normalize=True,
                          title=None,
                          rotate_xlabel=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
#     if not title:
#         if normalize:
#             title = 'Normalized confusion matrix'
#         else:
#             title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    if rotate_xlabel:
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    
    # save figure
    if title:
        plt.savefig(title, dpi=300, bbox_inches="tight")
    
    return ax

def plotCorrelationMatrix(df, title=None):
    # Create Correlation df
    corr = df.corr()
    # Plot figsize
    fig, ax = plt.subplots(figsize=(10, 10))

    # Drop self-correlations
    dropSelf = np.zeros_like(corr)
    dropSelf[np.triu_indices_from(dropSelf)] = True
    # Generate Heat Map, allow annotations and place floats in map
    sns.heatmap(corr, cmap="Blues", annot=True, fmt=".2f", mask=dropSelf, square=True)
    # Apply xticks
    plt.xticks(range(len(corr.columns)), corr.columns, size = 16)
    # Apply yticks
    plt.yticks(range(len(corr.columns)), corr.columns, size = 16)
    
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    
    # save figure
    if title:
        plt.savefig(title, dpi=300, bbox_inches="tight")
    
    # show plot
    plt.show()

    
def plotFeatureImportance(feature_importance, names, title = None):
    ax = plt.gca()
    plt.bar(range(len(feature_importance)), feature_importance)
    plt.yticks(fontsize=16)
    plt.xticks(range(len(feature_importance)), names.values, size = 16)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
    if title:
        plt.savefig(title, dpi=300, bbox_inches="tight")
    

# Linear Regression
- Best Subset, Forward Selection, or Lasso? (Ryan J. Tibshirani)
    - Best subset selection and forward selection usually perform similarly.
    - Lasso is preferred if low signal-to-noise ratio (SNR); Best subset if high SNR.

### Linear Regression, Vanilla

### Linear Regression, Lasso

In [ ]:
from sklearn.linear_model import Lasso

# y = energy_grad.iloc[:, 0]
# X = energy_grad.iloc[:, 1:]

# X_da = pd.concat([energy_grad.iloc[:, 1], geo_topo.iloc[:, :5], other.iloc[:, 1:]], axis=1)
# da = geo_topo.iloc[:, 5]
# y = da
# X = X_da

X_dfms = pd.concat([energy_grad.iloc[:, 1], geo_topo.iloc[:, :5], other.iloc[:, 1:]], axis=1)
dfms = geo_topo.iloc[:, 6]
y = dfms
# X = X_dfms
X = X_dfms / X_dfms.max(axis=0)


lasso = Lasso(alpha=1)

lasso.fit(X, y)
print lasso.coef_
print lasso.intercept_
print 'R2 score: ', lasso.score(X, y)
# print 'accuracy: ', metrics.accuracy_score(lasso.predict(X), y)


In [ ]:
# plotCorrelationMatrixHalf(pd.concat([dfms, X_dfms, geo_topo.C_diff], axis=1), 'dFMs_corr.png')
plotFeatureImportance(X_dfms.max(axis=0).values, X.columns, 'X_dfms_max_values')

### Linear Regression, Forward Selection
- Refs:
    - https://planspace.org/20150423-forward_selection_with_statsmodels/
    - https://xavierbourretsicotte.github.io/subset_selection.html